In [1]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
import os
from scipy.stats import zscore
import pyret.filtertools as ft
import pyret.spiketools as sp
import pyret.stimulustools as st
from deepretina.toolbox import load_model, load_partial_model, rolling_window
from deepretina.io import despine
from deepretina.metrics import cc

%matplotlib inline
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

plt.style.use('deepretina')

from pylab import rcParams
rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'seismic'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Using gpu device 0: GeForce GT 750M (CNMeM is disabled)
Using Theano backend.


You must install GPy (pip install GPy) to fit the GP regression nonlinearity.


In [7]:
data_dir = os.path.expanduser('~/data/kastner/')
horizontals = [x[0] for x in os.walk(data_dir + 'horizontal')]
amacrines = [x[0] for x in os.walk(data_dir + 'amacrine')]
bipolars = [x[0] for x in os.walk(data_dir + 'bipolar') if 'reproduce' not in x[0]]

### Take shortest amount of data so you can store all interneurons as an array with the same time

In [ ]:
boxes_len = None
intraRF_v_len = None
lines_len = None
for idh,h_file in enumerate(horizontals):
    try:
        boxes = np.loadtxt(h_file + '/' + 'boxes.txt', skiprows=1)
        boxes = boxes.reshape((-1, 38, 38))
        if (not boxes_len) or (boxes.shape[0] < boxes_len):
            boxes_len = boxes.shape[0]
            
        intraRF_v = np.loadtxt(h_file + '/' + 'intraRF_v.txt', skiprows=1)
        if (not intraRF_v_len) or (len(intraRF_v) < intraRF_v_len):
            intraRF_v_len = len(intraRF_v)
    
        lines = np.loadtxt(h_file + '/' + 'lines.txt', skiprows=1)
        lines = lines.reshape((-1, 100))
        if (not lines_len) or (lines.shape[0] < lines_len):
            lines_len = lines.shape[0]

    except:
        print(h_file)

In [ ]:
boxes_len

In [ ]:
intraRF_v_len

In [ ]:
lines_len